In [1]:
import numpy as np

PI = np.array([0.5,0.5])
A = np.array([[0.2,0.8],
             [0.999,0.001]])
B = np.array([[0.05,0.35,0.2,0.2,0.2],
             [0.5,0.1,0.3,0.1,0.0]])
Observables = np.array([0,3,0,3,0])

In [2]:
def forward(PI,A,B,Observables):
    alpha = np.zeros(shape=(2,len(Observables)))
    
#     Initialize first alpha values
    for i in range(len(PI)):
        alpha[i,0] = PI[i]*B[i][0]
   
    def forward_helper(j, t):
        sum = 0
        for i in range(len(PI)):
            sum += alpha[i, t-1]*A[j][i]
            
        return sum
    
    for t in range(1, len(Observables)):
        for j in range(len(PI)):
            alpha[j, t] = forward_helper(j, t)*B[j][Observables[t]]
             
    return alpha

In [3]:
forward(PI,A,B,Observables)

array([[2.50000000e-02, 4.10000000e-02, 5.10900000e-04, 3.29735780e-03,
        3.50970574e-05],
       [2.50000000e-01, 2.52250000e-03, 2.04807613e-02, 5.30869861e-05,
        1.64705676e-03]])

In [4]:
def forward_normalized(PI,A,B,Observables):
    alpha = np.zeros(shape=(2,len(Observables)))
    c = np.zeros(len(Observables))
#     Initialize first alpha values
    temp_sum = 0
    for i in range(len(PI)):
        alpha[i,0] = PI[i] * B[i][0]
        temp_sum += alpha[i,0]
    c[0] = 1/temp_sum
   
    def forward_helper(j, t):
        sum = 0
        for i in range(len(PI)):
            sum += alpha[i, t-1] * A[j][i]
            
        return sum
    
    for t in range(1, len(Observables)):
        temp_sum = 0
        for j in range(len(PI)):
            alpha[j,t] = forward_helper(j, t) * B[j][Observables[t]]
            temp_sum += alpha[j,t]
        c[t] = 1 / temp_sum
    
    for i in range(len(alpha)):
        alpha[i,:] = alpha[i,:] * c
    
    return alpha, c

In [5]:
alpha, c = forward_normalized(PI,A,B,Observables)
print("alpha: ", alpha)
print("c: ", c)

alpha:  [[0.09090909 0.94204147 0.02433824 0.98415524 0.02086436]
 [0.90909091 0.05795853 0.97566176 0.01584476 0.97913564]]
c:  [  3.63636364  22.97662129  47.63796386 298.46783452 594.47595511]


In [6]:
def backward(PI,A,B,Observables):
    beta=np.zeros(shape=(2,len(Observables)))
    
    for i in range(len(PI)):
        beta[i,len(Observables)-1] = 1
    
    def backward_helper(i, t):
        sum = 0
        for j in range(len(PI)):
            sum += B[j][Observables[t+1]] * beta[j,t+1] * A[i][j]
        return sum

    for t in range(len(Observables)-2,-1,-1):
        for i in range(len(PI)):
            beta[i, t] = backward_helper(i, t)

    return beta

In [7]:
backward(PI,A,B,Observables)

array([[0.00140388, 0.03297358, 0.020436  , 0.41      , 1.        ],
       [0.00658823, 0.00106174, 0.08192305, 0.05045   , 1.        ]])

In [8]:
def backward_normalized(PI,A,B,c,Observables):
    beta=np.zeros(shape=(2,len(Observables)))
    
    
    for i in range(len(PI)):
        beta[i,len(Observables)-1] = 1
    
    def backward_helper(i, t):
        sum = 0
        for j in range(len(PI)):
            sum += B[j][Observables[t+1]] * beta[j,t+1] * A[i][j]
        return sum

    for t in range(len(Observables)-2,-1,-1):
        for i in range(len(PI)):
            beta[i, t] = backward_helper(i, t)

    for i in range(len(beta)):
        beta[i,:] = beta[i,:] * c
    
    return beta

In [9]:
beta = backward_normalized(PI,A,B,c,Observables)
print("beta: ", beta)

beta:  [[5.10502654e-03 7.57621414e-01 9.73529429e-01 1.22371812e+02
  5.94475955e+02]
 [2.39571893e-02 2.43951915e-02 3.90264706e+00 1.50577023e+01
  5.94475955e+02]]


In [10]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    smoothed_matrix = np.zeros(shape=(noOfStates, noOfTimeSteps))
    total_sum = np.zeros(noOfTimeSteps)

    for i in range(noOfStates):
        smoothed_matrix[i] = alpha[i,:] * beta[i, :]
        total_sum += smoothed_matrix[i] 
    
    return  smoothed_matrix / total_sum

In [11]:
smoothed_probability(alpha,beta,len(PI),len(Observables))

array([[0.02086436, 0.99802285, 0.00618423, 0.99802285, 0.02086436],
       [0.97913564, 0.00197715, 0.99381577, 0.00197715, 0.97913564]])